In [24]:
import qiskit
import cirq
import braket
import numpy as np
from qbraid.transpiler.transpiler import qbraid_wrapper

In [25]:
qiskit_circ = qiskit.QuantumCircuit(4)

qiskit_circ.h(0)
qiskit_circ.h(1)
qiskit_circ.h(2)
qiskit_circ.h(3)
qiskit_circ.x(0)
qiskit_circ.x(1)
qiskit_circ.y(2)
qiskit_circ.z(3)
qiskit_circ.s(0)
qiskit_circ.sdg(1)
qiskit_circ.t(2)
qiskit_circ.tdg(3)
qiskit_circ.rx(np.pi/4, 0)
qiskit_circ.ry(np.pi/2, 1)
qiskit_circ.rz(3*np.pi/4, 2)
qiskit_circ.p(np.pi/8, 3)
qiskit_circ.sx(0)
qiskit_circ.sxdg(1)
qiskit_circ.iswap(2, 3)
qiskit_circ.swap(0, 2)
qiskit_circ.swap(1, 3)
qiskit_circ.cx(0, 1)
qiskit_circ.cp(np.pi/4, 2, 3)

qiskit_circ.draw()

┌───┐┌───┐ ┌───┐ ┌─────────┐   ┌────┐                 
q_0: ┤ H ├┤ X ├─┤ S ├─┤ RX(π/4) ├───┤ √X ├───X────────■────
     ├───┤├───┤┌┴───┴┐├─────────┤  ┌┴────┴┐  │      ┌─┴─┐  
q_1: ┤ H ├┤ X ├┤ SDG ├┤ RY(π/2) ├──┤ √XDG ├──┼──X───┤ X ├──
     ├───┤├───┤└┬───┬┘├─────────┴┐┌┴──────┴┐ │  │   └───┘  
q_2: ┤ H ├┤ Y ├─┤ T ├─┤ RZ(3π/4) ├┤0       ├─X──┼──■───────
     ├───┤├───┤┌┴───┴┐└┬────────┬┘│  ISWAP │    │  │P(π/4) 
q_3: ┤ H ├┤ Z ├┤ TDG ├─┤ P(π/8) ├─┤1       ├────X──■───────
     └───┘└───┘└─────┘ └────────┘ └────────┘

In [26]:
q0 = cirq.LineQubit(3)
q1 = cirq.LineQubit(2)
q2 = cirq.LineQubit(1)
q3 = cirq.LineQubit(0)

cirq_circ = cirq.Circuit()

gates = [
    cirq.H(q0),
    cirq.H(q1),
    cirq.H(q2),
    cirq.H(q3),
    cirq.X(q0),
    cirq.X(q1),
    cirq.Y(q2),
    cirq.Z(q3),
    cirq.S(q0),
    cirq.ZPowGate(exponent=-0.5)(q1),
    cirq.T(q2),
    cirq.ZPowGate(exponent=-0.25)(q3),
    cirq.Rx(rads=np.pi/4)(q0),
    cirq.Ry(rads=np.pi/2)(q1),
    cirq.Rz(rads=3*np.pi/4)(q2),
    cirq.ZPowGate(exponent=1/8)(q3),
    cirq.XPowGate(exponent=0.5)(q0), 
    cirq.XPowGate(exponent=-0.5)(q1),
    cirq.ISWAP(q2, q3),
    cirq.SWAP(q0, q2),
    cirq.SWAP(q1, q3),
    cirq.CNOT(q0, q1),
    cirq.CZPowGate(exponent=0.25)(q2, q3),
]

for gate in gates:
    cirq_circ.append(gate)

print(cirq_circ)

                                          ┌──┐
0: ───H───Z───T^-1───Z^(1/8)─────iSwap──────×────@────────
                                 │          │    │
1: ───H───Y───T──────Rz(0.75π)───iSwap─────×┼────@^0.25───
                                           ││
2: ───H───X───S^-1───Ry(0.5π)────X^-0.5────┼×────X────────
                                           │     │
3: ───H───X───S──────Rx(0.25π)───X^0.5─────×─────@────────
                                          └──┘


In [27]:
cirq_unitary = cirq_circ.unitary()
qiskit_unitary = qiskit.quantum_info.Operator(qiskit_circ).data

In [28]:
np.allclose(cirq_unitary, qiskit_unitary)

True

In [29]:
braket_circ = braket.circuits.Circuit()

braket_circ.h(0)
braket_circ.h(1)
braket_circ.h(2)
braket_circ.h(3)
braket_circ.x(0)
braket_circ.x(1)
braket_circ.y(2)
braket_circ.z(3)
braket_circ.s(0)
braket_circ.si(1)
braket_circ.t(2)
braket_circ.ti(3)
braket_circ.x(0)
braket_circ.x(1)
braket_circ.y(2)
braket_circ.z(3)
braket_circ.rx(0, np.pi/4)
braket_circ.ry(1, np.pi/2)
braket_circ.rz(2, 3*np.pi/4)
braket_circ.phaseshift(3, np.pi/8)
braket_circ.v(0)
braket_circ.vi(1)
braket_circ.iswap(2, 3)
braket_circ.swap(0, 2)
braket_circ.swap(1, 3)
braket_circ.cnot(0, 1)
braket_circ.cphaseshift(2, 3, np.pi/4)

print(braket_circ)

T  : |0|1|2 |3|     4      |  5  |    6    |     7      |
                                                         
q0 : -H-X-S--X-Rx(0.785)----V-----SWAP------C------------
                                  |         |            
q1 : -H-X-Si-X-Ry(1.57)-----Vi----|----SWAP-X------------
                                  |    |                 
q2 : -H-Y-T--Y-Rz(2.36)-----ISWAP-SWAP-|----C------------
                            |          |    |            
q3 : -H-Z-Ti-Z-PHASE(0.393)-ISWAP------SWAP-PHASE(0.785)-

T  : |0|1|2 |3|     4      |  5  |    6    |     7      |


In [38]:
braket_gates = ["H", "H", "H", "H"]
braket_operator = braket.circuits.quantum_operator.QuantumOperator(4, braket_gates)
# https://amazon-braket-sdk-python.readthedocs.io/en/latest/_apidoc/braket.circuits.quantum_operator.html

In [43]:
assert False

AssertionError: 